In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
import base64
import hashlib
import unicodedata

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from typing import List, Dict
from typing import Callable
from datetime import datetime

In [ ]:
# Funções de conversão
def remover_acentos_e_transformar_minusculo(texto):
    # Transforma o texto em minúsculas
    texto = texto.lower()
    # Normaliza a string para remover acentos
    texto_sem_acentos = unicodedata.normalize('NFD', texto)
    texto_sem_acentos = texto_sem_acentos.encode('ascii', 'ignore').decode('utf-8')
    return texto_sem_acentos


def substituir_string(texto, lista_substituicoes):  
    for padrao, novo in lista_substituicoes:
        if padrao.startswith('^'):
            if not padrao[1:] in texto: 
                texto = novo
        else:
            texto = texto.replace(padrao, novo)
    return texto

def string_to_unique_float2(s: str) -> float:
    # Gera um hash MD5 da string
    md5_hash = hashlib.md5(s.encode()).hexdigest()
    # Converte o hash em um número inteiro
    int_value = int(md5_hash, 16)
    # Normaliza o valor para o intervalo entre [0,1] e retorna
    return int_value % (10**8) / (10**8)

def string_to_unique_float(texto: str) -> float:
    texto_bytes = texto.encode('utf-8')
    numero_base64 = base64.b64encode(texto_bytes)
    numero_unico = int.from_bytes(numero_base64, byteorder='big')
    max_valor = 2 ** (len(numero_base64) * 8) - 1  # Maior valor possível para o número
    return numero_unico / max_valor

def replace_with_other(dataResp: Dict[str, list[str]],colNum: int, text,list: List,func: Callable[[str], str] = string_to_unique_float):
        text = substituir_string(text,list)
        if text.startswith("other:") :
            appendData(dataResp, f'Resp_{colNum}', func(text.replace("other:","")))
        else:
            appendData(dataResp, f'Resp_{colNum}', float(text))             

def appendData(dataResp: Dict[str, list[str]], key: str, colData: str):
    try:
        dataResp[key].append(colData)
    except KeyError:
        dataResp[key] = [colData]


def trataCol(dataResp: Dict[str, list[str]], colNum: int, colData: str, timeAtual: pd.Timestamp, timeAnterior: pd.Timestamp):
    enc = OneHotEncoder(sparse_output=False)
    timeElipsed = (timeAtual - timeAnterior).total_seconds()    
    colData = colData.replace("other;", "other:").replace("Sucess;", "").replace("Sucess", "")
    match colNum:
        case 2:
            for index,item in enumerate(colData.split(";")):
                match index:
                    case 0: # Que horas são neste instante?
                        hora = item
                    case 1: # Data de hoje?
                        # Calcula o quanto o entrevistado estava errado em noção do tempo
                        appendData(dataResp, 'Resp_1', (timeAtual - pd.to_datetime(f"{item} {hora}", dayfirst=True)).total_seconds())
                    case 2: # Qual a sua Idade?
                        appendData(dataResp, 'Resp_2', item)                       
                    case 3: # [Categórica] Gênero * - 
                        appendData(dataResp, 'Resp_3', enc.fit_transform(item))  
                        # replace_with_other(dataResp,index,item,[("Feminino", "0.0"), ("Masculino", "1.0")])                         
                    case 4: # [Categórica] Qual foi o sexo atribuído no seu nascimento?
                        replace_with_other(dataResp,index,item,[("Feminino", "0.0"), ("Masculino", "1.0")])
                    case 5: # [Categórica] Assinale a alternativa que identifica a sua Cor ou Raça:
                        appendData(dataResp, f'Resp_{index}', string_to_unique_float(item.replace("other:","")))                        
                        # replace_with_other(dataResp,index,item,[("Preta", "0"), ("Branca", "1"), ("Parda", "2"), ("Amarela", "3"), ("IndÍgena", "4")])
                    case 6: # Dentro de sua família, você é o(a) único(a) filho(a)?
                        replace_with_other(dataResp,index,item,[("Sim", "0.0")],lambda x: float(x))
                    case 7: # [Categórica] Qual o seu estado civil?
                        appendData(dataResp, f'Resp_{index}', string_to_unique_float(item.replace("other:","")))   
                    case 8: # Possui filhos(as)?
                        replace_with_other(dataResp,index,item,[("Não", "0.0")],lambda x: float(x)) 
                    case 9: # Possui filhos(as) menores de 6 anos?
                        replace_with_other(dataResp,index,item,[("Não", "0.0"), ("Sim", "1.0")])    
                    case 10: # [Categórica] Religião *
                        replace_with_other(dataResp,index,item,[("Sem religião", "0.0")])
                    case 11: # [Categórica] Escolaridade *
                        appendData(dataResp, 'Resp_11', enc.fit_transform(item))  
                        # replace_with_other(dataResp,index,item,[
                        #     ("Sem Escolaridade", "0.0"),
                        #     ("Ensino Fundamental (1º grau) incompleto","0.1"),
                        #     ("Ensino Fundamental (1º grau) completo", "0.2"),
                        #     ("Ensino Médio (2º grau) incompleto", "0.3"),
                        #     ("Ensino Médio (2º grau) completo", "0.4"),
                        #     ("Superior Incompleto", "0.55"),
                        #     ("Superior Completo", "0.7"),
                        #     ("Mestrado", "0.85"),
                        #     ("Doutorado", "1.0")
                        # ])
                    case 12: # [Categórica] Renda familiar mensal de sua casa (somatória)
                        replace_with_other(dataResp,index,item,[
                            ("Até 1 salário mínimo", "0.0"),
                            ("Mais de 1 a 2 salários mínimos", "0.15"),
                            ("Mais de 2 a 3 salários mínimos", "0.3"),
                            ("Mais de 3 a 5 salários mínimos", "0.45"),
                            ("Mais de 5 a 8 salários mínimos", "0.6"),
                            ("Mais de 8 a 12 salários mínimos", "0.75"),
                            ("Mais de 12 a 20 salários mínimos", "9.0"),
                            ("Mais de 20 salários mínimos", "1.0")
                        ])                     
                appendData(dataResp, f'Elipsed_{index}', timeElipsed)                            
        case 3: # [Categórica] Target
            appendData(dataResp, 'Resp_13', enc.fit_transform(item)) 
            # replace_with_other(dataResp,13,colData,[
            #         (";", ""),                
            #         ("Não", "0.0"),
            #         ("other:Transtorno do Espectro Autista", "1.0"),
            #         ("other:Transtornos Depressivos", "2.0"),
            #         ("other:Transtorno Ciclotímico", "3.0"),
            #         ("other:Transtornos de Ansiedade", "4.0"),
            #         ("other:Transtorno de Estresse Pós-traumático", "5.0"),
            #         ("other:Transtornos Alimentares", "6.0"),
            #         ("other:Transtorno Bipolar", "7.0"),
            #         ("other:Transtorno Obsessivo-compulsivo", "8.0"),
            #         ("other:Transtorno de Déficit de Atenção/Hiperatividade", "9.0"),
            #         ("other:Transtorno da Personalidade Borderline", "10.0"),
            #         ("other:Transtorno do Espectro da Esquizofrenia e Outros Transtornos Psicóticos", "11.0")
            #     ])
            appendData(dataResp, f'Elipsed_{13}', timeElipsed)  
        case 7: # [Categórica]
            replace_with_other(dataResp,14,colData,[
              ("Jesus Cristo", "0.0"),
              ("Coração", "0.05"),
              ("Dragão cuspindo fogo", "0.01"),
              ("Árvore", "1.0"),
              ("Não vi nada", "0.1"),
              ("Outra coisa", "0.15")
            ])           
            appendData(dataResp,'Elipsed_14', timeElipsed)               
        case 10: # [Categórica] 
            replace_with_other(dataResp,15,colData,[
              ("Jesus Cristo", "0.0"),
              ("Coração", "0.05"),
              ("Dragão cuspindo fogo", "0.01"),
              ("Árvore", "0.1"),
              ("Não vi nada", "1.0"),
              ("Outra coisa", "0.15")
            ])           
            appendData(dataResp,'Elipsed_15', timeElipsed)
        case 13:  # [Categórica] Qual foi a sequência correta dos números apresentados na tela anterior?
            replace_with_other(dataResp,16,colData,[
              ("1 - 5", "0.1"),
              ("4 - 7", "0.2"),
              ("2 - 7", "1.0"),
              ("2 - 8", "0.6"),
              ("9 - 4", "0.4"),
              ("7 - 2", "0.7")
            ])           
            appendData(dataResp, 'Elipsed_16', timeElipsed)
        case 15:  # [Categórica] Qual foi a sequência correta dos números apresentados na tela anterior?
            replace_with_other(dataResp,17,colData,[
              ("5 - 7 - 1", "0.7"),
              ("1 - 3 - 4", "0.1"),
              ("5 - 6 - 3", "0.8"),
              ("4 - 6 - 5", "0.2"),
              ("5 - 4 - 6", "0.9"),
              ("5 - 6 - 4", "1.0")
            ])           
            appendData(dataResp, 'Elipsed_17', timeElipsed)
        case 17: # Qual foi a sequência correta dos números apresentados na tela anterior?
            replace_with_other(dataResp,18,colData,[
              ("6 - 4 - 9 - 3", "0.8"),
              ("4 - 8 - 9 - 1", "0.3"),
              ("6 - 4 - 3 - 9", "1.0"),
              ("5 - 4 - 3 - 8", "0.4"),
              ("1 - 5 - 2 - 9", "0.2"),
              ("6 - 4 - 3 - 7", "0.7")
            ])           
            appendData(dataResp, 'Elipsed_18', timeElipsed)
        case 19: # Qual foi a sequência correta dos números apresentados na tela anterior?
            replace_with_other(dataResp,19,colData,[
              ("2 - 1 - 4 - 7 - 9", "0.4"),
              ("4 - 3 - 9 - 8", "0.5"),
              ("4 - 2 - 6 - 3 - 1", "0.8"),
              ("7 - 5 - 1 - 4 - 2 ", "0.3"),
              ("4 - 2 - 7 - 3 - 1", "1.0"),
              ("6 - 3 - 1 - 5 - 9", "0.2")
            ])           
            appendData(dataResp, 'Elipsed_19', timeElipsed)
        case 21: # Qual foi a sequência correta dos números apresentados na tela anterior?
            replace_with_other(dataResp,20,colData,[
              ("6 - 1 - 4 - 7 - 3 - 9", "0.7"),
              ("2 - 1 - 8 - 3 - 9 - 5", "0.4"),
              ("6 - 1 - 9 - 4 - 7 - 3", "1.0"),
              ("6 - 4 - 5 - 8 - 3 - 7", "0.5"),
              ("2 - 8 - 6 - 4 - 7 - 3", "0.3"),
              ("6 - 1 - 9 - 4 - 5 - 2", "0.8")
            ])           
            appendData(dataResp, 'Elipsed_20', timeElipsed)
        case 23: # Qual foi a sequência correta dos números apresentados na tela anterior?
            replace_with_other(dataResp,21,colData,[
              ("5 - 8 - 1 - 7 - 4 - 3 - 8", "0.7"),
              ("5 - 9 - 1 - 7 - 4 - 8", "0.8"),
              ("8 - 9 - 0 - 7 - 3 - 1", "0.1"),
              ("5 - 9 - 1 - 7 - 4 - 2 - 8", "1.0"),
              ("5 - 2 - 3 - 7 - 4 - 9 - 8", "0.6"),
              ("5 - 9 - 1 - 7 - 8 - 0 - 9", "0.5")
            ])           
            appendData(dataResp, 'Elipsed_21', timeElipsed)
        case 25: # Dentre as quatro alternativas de cada imagem. Selecione a palavra que melhor a descreve
            for index,item in enumerate(colData.split(";")):
                match index:
                    case 0: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Inquieto(a)', "0.1"),
                            ('Pensativo(a)', "1.0"),
                            ('Irritado(a)', "0.2"),
                            ('Desconfiado(a)', "0.3")
                        ])
                    case 1: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Nervoso(a)', "0.1"),
                            ('Deprimido(a)', "0.2"),
                            ('Irritado(a)', "0.3"),
                            ('Divertido(a)', "1.0")                            
                        ])
                    case 2: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Envergonhado(a)', "0.1"),
                            ('Divertido(a)', "0.2"),
                            ('Interessado(a)', "1.0"),
                            ('Deprimido(a)', "0.3")                            
                        ])                      
                    case 3: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Arrogante', "0.3"),
                            ('Decidido(a)', "0.2"),
                            ('Apavorado(a)', "1.0"),
                            ('Chateado(a)', "0.1")                            
                        ])                        
                    case 4: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Amável', "0.1"),
                            ('Decidido(a)', "0.2"),
                            ('Simpático(a)', "0.3"),
                            ('Deprimido(a)', "1.0")                            
                        ])
                    case 5: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Tímido(a)', "0.1"),
                            ('Perturbado(a)', "1.0"),
                            ('Desanimado(a)', "0.2"),
                            ('Pensativo(a)', "0.3")                            
                        ])                        
                    case 6: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Impaciente', "1.0"),
                            ('Desanimado(a)', "0.2"),
                            ('Sedutor(a)', "0.1"),
                            ('Aliviado(a)', "0.3")                            
                        ])
                    case 7: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Grato(a)', "0.3"),
                            ('Sonhador(a)', "0.1"),
                            ('Desanimado(a)', "1.0"),
                            ('Chocado(a)', "0.2")                            
                        ])  
                    case 8: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Satisfeito(a)', "0.3"),
                            ('Preocupado(a)', "1.0"),
                            ('Afetuoso(a)', "0.1"),
                            ('Indiferente', "0.2")                            
                        ]) 
                    case 9: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Amável', "0.1"),
                            ('Arrependido(a)', "1.0"),
                            ('Zangado(a)', "0.2"),
                            ('Simpático(a)', "0.3")                            
                        ])   
                    case 10: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Desconfortável', "1.0"),
                            ('Entediado(a)', "0.1"),
                            ('Confiante', "0.2"),
                            ('Impaciente', "0.3")                            
                        ])
                    case 11: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Arrependido(a)', "0.2"),
                            ('Nervoso(a)', "1.0"),
                            ('Divertido(a)', "0.3"),
                            ('Envergonhado(a)', "0.1")                            
                        ])
                    case 12: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Amigável', "0.1"),
                            ('Entretido(a)', "0.2"),
                            ('Desconfiado(a)', "1.0"),
                            ('Sedutor(a)', "0.3")                            
                        ]) 
                    case 13: # 
                        replace_with_other(dataResp,22 + index,item,[
                            ('Preocupado(a)', "0.2"),
                            ('Cauteloso(a)', "0.1"),
                            ('Hostil', "1.0"),
                            ('Divertido(a)', "0.3")                            
                        ])                                              
                appendData(dataResp, f'Elipsed_{22 + index}', timeElipsed) 
        case 26: # Responda as questões abaixo:
            colData = re.sub(r'<optimized out>#\w+\((Sim|Não)\)', r'\1', colData)              
            for index,item in enumerate(colData.split(";")):
                replace_with_other(dataResp,36 + index,item,[
                    ('Não', "0.0"),
                    ('Sim', "1.0"),
                ])                                  
                appendData(dataResp, f'Elipsed_{36 + index}', timeElipsed) 
        case 27: # Qual das imagens abaixo completa a sequência a seguir?
            replace_with_other(dataResp,42,colData,[
              (";", ""),
              ("assets/intel_1", ""),
              ("a", "0.1"),
              ("b", "0.2"),
              ("c", "0.3"),
              ("d", "0.4"),
              ("e", "1.0"),
              ("f", "0.5")            
            ])           
            appendData(dataResp, 'Elipsed_42', timeElipsed)
        case 28: # Responda as questões abaixo:
            colData = re.sub(r'<optimized out>#\w+\((Sim|Não)\)', r'\1', colData)       
            for index,item in enumerate(colData.split(";")):
                replace_with_other(dataResp,43 + index,item,[
                    ('Não', "0.0"),
                    ('Sim', "1.0"),
                ])                                    
                appendData(dataResp, f'Elipsed_{43 + index}', timeElipsed)
        case 29: # Complete as sequências a seguir:
            for index,item in enumerate(colData.split(";")):
                appendData(dataResp, f'Resp_{50 + index}', float(item))                                                                          
                appendData(dataResp, f'Elipsed_{50 + index}', timeElipsed) 
        case 30: # Observe as palavras a seguir:
            replace_with_other(dataResp,54,colData,[
              ("1 - 4 - 6 - 2 - 5 - 3 - 7", "0.4"),
              ("6 - 2 - 4 - 7 - 5 - 1 - 3", "0.2"),
              ("5 - 1 - 3 - 4 - 6 - 7 - 2", "0.1"),
              ("6 - 2 - 5 - 1 - 3 - 4 - 7", "1.0"),
              ("7 - 4 - 3 - 5 - 1 - 2 - 6", "0.3"),          
            ])           
            appendData(dataResp, 'Elipsed_54', timeElipsed)
        case 31: # Responda as questões abaixo: + 6
            colData = re.sub(r'<optimized out>#\w+\((Sim|Não)\)', r'\1', colData)       
            for index,item in enumerate(colData.split(";")):
                replace_with_other(dataResp,55 + index,item,[
                    ('Não', "0.0"),
                    ('Sim', "1.0"),
                ])                                    
                appendData(dataResp, f'Elipsed_{55 + index}', timeElipsed)  
        case 32: # Preencha o campo a seguir com o nome da cidade e estado onde você está agora.
            item = colData.split(";")      
            appendData(dataResp, 'Resp_61', float(string_to_unique_float(item[0]))) # CIDADE:                                                                        
            appendData(dataResp, 'Elipsed_61', timeElipsed) 
            
            appendData(dataResp, 'Resp_62', float(string_to_unique_float(item[1]))) # ESTADO:                                                                        
            appendData(dataResp, 'Elipsed_62', timeElipsed)
        case 33: # Preencha o campo a seguir com o nome da cidade e estado onde você está agora.
            hora = timeAtual.hour           
            replace_with_other(dataResp,63,colData,[
              ("Manhã: 6:00 às 11:59 horas", "1.0" if 6 <= hora < 12 else "0.0"),
              ("Tarde: 12:00 às 17:59 horas", "1.0" if 12 <= hora < 18 else "0.0"),
              ("Noite: 18:00 às 23:59 horas", "1.0" if 18 <= hora < 0 else "0.0"),
              ("Madrugada: 00:00 às 05:59 horas", "1.0" if 0 <= hora < 6 else "0.0")    
            ])           
            appendData(dataResp, 'Elipsed_63', timeElipsed) 
        case 41: # + 7
            colData = re.sub(r'<optimized out>#\w+\((Sim|Não)\)', r'\1', colData)       
            for index,item in enumerate(colData.split(";")):
                replace_with_other(dataResp,64 + index,item,[
                    ('Não', "0.0"),
                    ('Sim', "1.0"),
                ])                                    
                appendData(dataResp, f'Elipsed_{64 + index}', timeElipsed) 
        case 42: # Preencha o campo a seguir com o nome da cidade e estado onde você está agora.
            index = 0.0
            for item in colData.split(";"):
                str = item.split(",")
                if 43.0 < float(str[0]) < 64.0 and 393.0 < float(str[1]) < 414.0: index += 1.0 #  53.799999999999955, 403.8 
                if 422.0 < float(str[0]) < 443.0 and 38.0 < float(str[1]) < 59.0: index += 1.0 # 432.79999999999995,   48.80000000000001;                
                if 28.0 < float(str[0]) < 49.0 and 201.0 < float(str[1]) < 222.0: index += 1.0 #  38.799999999999955, 211.8;                
                if 105.0 < float(str[0]) < 126.0 and 13.0 < float(str[1]) < 34.0: index += 1.0 # 115.79999999999995,   23.80000000000001;                
                if 274.0 < float(str[0]) < 295.0 and 369.0 < float(str[1]) < 390.0: index += 1.0 # 284.79999999999995,  379.8;                
                if 341.0 < float(str[0]) < 362.0 and 232.0 < float(str[1]) < 253.0: index += 1.0 # 351.79999999999995,  242.8                                                          
            appendData(dataResp, 'Resp_71', float(index)/6.0)                                                                         
            appendData(dataResp, 'Elipsed_71', timeElipsed)                 
        case 43: # Preencha o campo a seguir com o nome da cidade e estado onde você está agora.
            index = 0
            for item in colData.split(";"):
              if item == 'Encontrar-se com um ente querido falecido.': index += 1
              if item == 'Ver uma pessoa nua': index += 2
              if item == 'Saber como desempenhar tarefas cotidianas sem ser prejudicado por pensamentos incessantes que me limitam.': index += 4
              if item == 'Voltar na minha infância e recomeçar tudo': index += 8
              if item == 'Voltar na minha adolescência e recomeçar tudo': index += 16
              if item == 'Saber se meu namorado (a) ou esposo (a) está me traindo': index += 32
              if item == 'Saber como poderia ser o futuro da minha família e ajudá-los': index += 64
              if item == 'Desaparecer no tempo e espaço, ao entrar pela fenda': index += 128
              if item == 'Saber se estarei vivo daqui 5 anos.': index += 256
              if item == 'Saber quem é minha alma gêmea': index += 512
              if item == 'Ver meu futuro profissional': index += 1024
              if item == 'Saber como faço para não pensar coisas bizarras': index += 2048
              if item == 'Testemunhar avanços tecnológicos futuros.': index += 4096
              if item == 'Presenciar um evento histórico de grande relevância, como a construção das pirâmides do Egito, a Grande Muralha da China, a era dos dinossauros, ou a criação de alguma das invenções de Albert Einstein ou Leonardo da Vinci, por exemplo.': index += 8192
              if item == 'Saber se ficarei rico(a)': index += 16384
              if item == 'Saber quem está me perseguindo na rua': index += 32768
              if item == 'Saber como faço para dormir a noite toda.': index += 65536
              if item == 'Saber como fazer meu cônjuge e/ou filho(a) a realizar tarefas e tomar decisões em suas vidas de acordo com meus valores e princípios.': index += 131072
              if item == 'Saber como desaparecer minha dor e sofrimento existencial.': index += 262144
            appendData(dataResp, 'Resp_72', float(index)/524288.0)                                                                         
            appendData(dataResp, 'Elipsed_72', timeElipsed)                    
        case 44: # + 6
            colData = re.sub(r'<optimized out>#\w+\((Sim|Não)\)', r'\1', colData)       
            for index,item in enumerate(colData.split(";")):
                replace_with_other(dataResp,73 + index,item,[
                    ('Não', "0.0"),
                    ('Sim', "1.0"),
                ])                                    
                appendData(dataResp, f'Elipsed_{73 + index}', timeElipsed) 
        case 47: # Preencha o campo a seguir com o nome da cidade e estado onde você está agora.
            replace_with_other(dataResp,79,colData,[
              ("Pássaros", "0.4"),
              ("Barulho de água", "1.0"),
              ("Aspirador de pó", "0.1"),
              ("Choro de criança", "0.2"),
              ("Telefone tocando", "0.3"),  
              ("Sem som", "0.0"),                          
            ])           
            appendData(dataResp, 'Elipsed_79', timeElipsed)      
        case 48: # Preencha o campo a seguir com o nome da cidade e estado onde você está agora.
            replace_with_other(dataResp,80,colData,[
              ("A Reta A é maior que a Reta B", "0.4"),
              ("A Reta A é menor que a Reta B", "0.2"),
              ("Retas A e B, são do mesmo tamanho", "1.0"),                      
            ])           
            appendData(dataResp, 'Elipsed_80', timeElipsed)   
        case 49: # Preencha o campo a seguir com o nome da cidade e estado onde você está agora.
            item = colData.split(";")      
            appendData(dataResp, 'Resp_81', float(string_to_unique_float(item[0])))                                                                        
            appendData(dataResp, 'Elipsed_81', timeElipsed)                                
        case 50: # + 7
            colData = re.sub(r'<optimized out>#\w+\((Sim|Não)\)', r'\1', colData)       
            for index,item in enumerate(colData.split(";")):
                replace_with_other(dataResp,82 + index,item,[
                    ('Não', "0.0"),
                    ('Sim', "1.0"),
                ])                                    
                appendData(dataResp, f'Elipsed_{82 + index}', timeElipsed)
        case 51: # Preencha o campo a seguir com o nome da cidade e estado onde você está agora.
            index = 0.0
            for item in colData.split(";"):
                str = item.split(",")
                if 196.0 < float(str[0]) < 217.0 and 579.0 < float(str[1]) < 600.0: index += 1.0 # 206.79999999999995,589.8 
                if 105.0 < float(str[0]) < 126.0 and 585.0 < float(str[1]) < 606.0: index += 1.0 # 115.79999999999995,595.8   
                if 206.0 < float(str[0]) < 227.0 and 46.0 < float(str[1]) < 67.0: index += 1.0 # 216.79999999999995,56.80000000000001
                if 376.0 < float(str[0]) < 397.0 and 480.0 < float(str[1]) < 501: index += 1.0 # 386.79999999999995,490.8
                if 293.0 < float(str[0]) < 314.0 and 668.0 < float(str[1]) < 689.0: index += 1.0 # 303.79999999999995,678.3169811320754
            appendData(dataResp, 'Resp_89', float(index)/5.0)                                                                         
            appendData(dataResp, 'Elipsed_89', timeElipsed)                 
        case 52: # + 7
            colData = re.sub(r'<optimized out>#\w+\((Sim|Não)\)', r'\1', colData)       
            for index,item in enumerate(colData.split(";")):
                replace_with_other(dataResp,90 + index,item,[
                    ('Não', "0.0"),
                    ('Sim', "1.0"),
                ])                                    
                appendData(dataResp, f'Elipsed_{90 + index}', timeElipsed)   
        case 53: # Qual das imagens abaixo completa a sequência a seguir?
            replace_with_other(dataResp,97,colData,[
              (";", ""),
              ("assets/intel_2", ""),
              ("a", "0.1"),
              ("b", "1.0"),
              ("c", "0.3"),
              ("d", "0.4"),
              ("e", "0.2"),
              ("f", "0.5")            
            ])           
            appendData(dataResp, 'Elipsed_97', timeElipsed)                     
        case 54: # + 7
            colData = re.sub(r'<optimized out>#\w+\((Sim|Não)\)', r'\1', colData)       
            for index,item in enumerate(colData.split(";")):
                replace_with_other(dataResp,98 + index,item,[
                    ('Não', "0.0"),
                    ('Sim', "1.0"),
                ])                                    
                appendData(dataResp, f'Elipsed_{98 + index}', timeElipsed)  
        case 57: # Preencha o campo a seguir com o nome da cidade e estado onde você está agora.
            index = 0
            for item in colData.split(";"):
              if item == 'assets/questao45coquetel': index += 1
              if item == 'assets/questao45humburgue': index += 2
              if item == 'assets/questao45casa': index += 4
              if item == 'assets/questao45gato': index += 8
              if item == 'assets/questao45trabalhador': index += 16
              if item == 'assets/questao45carro': index += 32
              if item == 'assets/questao45cachorro': index += 64
              if item == 'assets/questao45passaro': index += 128
              if item == 'assets/questao45cerveja': index += 256
              if item == 'assets/questao45cocacola': index += 512
              if item == 'assets/questao45cafe': index += 1024
              if item == 'assets/questao45bombons': index += 2048
              if item == 'assets/questao45viajar': index += 4096
              if item == 'assets/questao45livros': index += 8192
              if item == 'assets/questao45leaonovo': index += 16384
              if item == 'assets/questao45controlegame': index += 32768
            appendData(dataResp, 'Resp_105', float(index)/65536.0)                                                                         
            appendData(dataResp, 'Elipsed_105', timeElipsed)                  
        case 58: # + 7
            colData = re.sub(r'<optimized out>#\w+\((Sim|Não)\)', r'\1', colData)       
            for index,item in enumerate(colData.split(";")):
                replace_with_other(dataResp,106 + index,item,[
                    ('Não', "0.0"),
                    ('Sim', "1.0"),
                ])                                    
                appendData(dataResp, f'Elipsed_{106 + index}', timeElipsed) 
        case 59: # Qual das opções corresponde ao som escutado?
            replace_with_other(dataResp,113,colData,[
              (";", ""),
              ("Rua - Madeira - Paz - Pastel", "0.1"),
              ("Lua - Cadeira - Raiz - Chapéu", "0.2"),
              ("Rua - Cadeira - Paz - Chapéu", "1.0"),
              ("Lua - Cadeira - Paz - Pastel", "0.4"),          
            ])           
            appendData(dataResp, 'Elipsed_113', timeElipsed)   
        case 60: # + 6 Preencha o campo a seguir com o nome da cidade e estado onde você está agora.
            for index,item in enumerate(colData.split(";")):
                match index:
                    case 0: # 
                        replace_with_other(dataResp,114 + index,item,[
                            (' ', "0.0"),
                            ('Nojo', "0.1"),
                            ('Tristeza', "0.2"),
                            ('Medo', "0.3"),
                            ('Raiva', "0.4"),
                            ('Alegria', "1.0"),
                            ('Surpresa', "0.5")                            
                        ])
                    case 1: # 
                        replace_with_other(dataResp,114 + index,item,[
                            (' ', "0.0"),
                            ('Nojo', "0.1"),
                            ('Tristeza', "0.2"),
                            ('Medo', "1.0"),
                            ('Raiva', "0.4"),
                            ('Alegria', "0.3"),
                            ('Surpresa', "0.5")                            
                        ])
                    case 2: # 
                        replace_with_other(dataResp,114 + index,item,[
                            (' ', "0.0"),
                            ('Nojo', "0.1"),
                            ('Tristeza', "0.2"),
                            ('Medo', "0.3"),
                            ('Raiva', "0.4"),
                            ('Alegria', "0.5"),
                            ('Surpresa', "1.0")                            
                        ])                      
                    case 3: # 
                        replace_with_other(dataResp,114 + index,item,[
                            (' ', "0.0"),
                            ('Nojo', "0.1"),
                            ('Tristeza', "1.0"),
                            ('Medo', "0.3"),
                            ('Raiva', "0.4"),
                            ('Alegria', "0.2"),
                            ('Surpresa', "0.5")                            
                        ])                       
                    case 4: # 
                        replace_with_other(dataResp,114 + index,item,[
                            (' ', "0.0"),
                            ('Nojo', "1.0"),
                            ('Tristeza', "0.2"),
                            ('Medo', "0.3"),
                            ('Raiva', "0.4"),
                            ('Alegria', "0.1"),
                            ('Surpresa', "0.5")                            
                        ])
                    case 5: # 
                        replace_with_other(dataResp,114 + index,item,[
                            (' ', "0.0"),
                            ('Nojo', "0.1"),
                            ('Tristeza', "0.2"),
                            ('Medo', "0.3"),
                            ('Raiva', "1.0"),
                            ('Alegria', "0.4"),
                            ('Surpresa', "0.5")                            
                        ])                                                                    
                appendData(dataResp, f'Elipsed_{114 + index}', timeElipsed)
        case 62: # Selecione pelo menos 2 e no máximo 4 expressões que melhor correspondem ao que você tem sentido nos últimos meses.
            index = 0
            for item in colData.split(";"):
              if item == 'assets/emoji_sempre_atrasado': index += 1
              if item == 'assets/emoji_poderoso': index += 2
              if item == 'assets/emoji_otimista': index += 4
              if item == 'assets/emoji_em_panico': index += 8
              if item == 'assets/emoji_indeciso': index += 16
              if item == 'assets/emoji_triste': index += 32
              if item == 'assets/emoji_entediado': index += 64
              if item == 'assets/emoji_pessimista': index += 128
              if item == 'assets/emoji_reflexivo': index += 256
              if item == 'assets/emoji_confiante': index += 512
              if item == 'assets/emoji_forte': index += 1024
              if item == 'assets/emoji_nojo': index += 2048
            appendData(dataResp, 'Resp_120', float(index)/4096.0)                                                                         
            appendData(dataResp, 'Elipsed_120', timeElipsed) 
        case 63: # Selecione pelo menos 2 e no máximo 4 expressões que melhor correspondem ao que você tem sentido nos últimos meses.
            index = 0
            for item in colData.split(";"):
              if item == 'assets/emoji_esperancoso': index += 1
              if item == 'assets/emoji_pura_alegria': index += 2
              if item == 'assets/emoji_burro': index += 4
              if item == 'assets/emoji_surpreso': index += 8
              if item == 'assets/emoji_feliz': index += 16
              if item == 'assets/emoji_velho': index += 32
              if item == 'assets/emoji_frustrado': index += 64
              if item == 'assets/emoji_ansioso': index += 128
              if item == 'assets/emoji_emocionado': index += 256
              if item == 'assets/emoji_em_paz': index += 512
              if item == 'assets/emoji_inteligente': index += 1024
              if item == 'assets/emoji_preocupado': index += 2048
            appendData(dataResp, 'Resp_121', float(index)/4096.0)                                                                         
            appendData(dataResp, 'Elipsed_121', timeElipsed)
        case 64: # Selecione pelo menos 2 e no máximo 4 expressões que melhor correspondem ao que você tem sentido nos últimos meses.
            index = 0
            for item in colData.split(";"):
              if item == 'assets/emoji_apaixonado': index += 1
              if item == 'assets/emoji_desesperado': index += 2
              if item == 'assets/emoji_envergonhado': index += 4
              if item == 'assets/emoji_abencoado': index += 8
              if item == 'assets/emoji_impulsivo': index += 16
              if item == 'assets/emoji_amado': index += 32
              if item == 'assets/emoji_confuso': index += 64
              if item == 'assets/emoji_sem_forcas': index += 128
              if item == 'assets/emoji_sonolento': index += 256
              if item == 'assets/emoji_gordo': index += 512
              if item == 'assets/emoji_sem_paciencia': index += 1024
              if item == 'assets/emoji_com_ciumes': index += 2048
            appendData(dataResp, 'Resp_122', float(index)/4096.0)                                                                         
            appendData(dataResp, 'Elipsed_122', timeElipsed)
        case 65: # Selecione pelo menos 2 e no máximo 4 expressões que melhor correspondem ao que você tem sentido nos últimos meses.
            index = 0
            for item in colData.split(";"):
              if item == 'assets/emoji_com_raiva': index += 1
              if item == 'assets/emoji_silenciado': index += 2
              if item == 'assets/emoji_desanimado': index += 4
              if item == 'assets/emoji_com_gratidao': index += 8
              if item == 'assets/emoji_fraude': index += 16
              if item == 'assets/emoji_agressivo': index += 32
              if item == 'assets/emoji_desequilibrado': index += 64
              if item == 'assets/emoji_comendo_muito': index += 128
              if item == 'assets/emoji_fumando_muito': index += 256
              if item == 'assets/emoji_jesus': index += 512
              if item == 'assets/emoji_bebendo_muito': index += 1024
              if item == 'assets/emoji_animado': index += 2048
            appendData(dataResp, 'Resp_123', float(index)/4096.0)                                                                         
            appendData(dataResp, 'Elipsed_123', timeElipsed)
        case 66: # Selecione pelo menos 2 e no máximo 4 expressões que melhor correspondem ao que você tem sentido nos últimos meses.
            index = 0
            for item in colData.split(";"):
              if item == 'ABANDONO': index += (1 << 0)
              if item == 'ACOMETIDO(A)': index += (1 << 1)
              if item == 'AFEIÇÃO': index += (1 << 2)
              if item == 'AMIUDAR': index += (1 << 3)
              if item == 'ANTIPATIA': index += (1 << 4)
              if item == 'APATIA': index += (1 << 5)
              if item == 'APEGO': index += (1 << 6)
              if item == 'APREENSÃO': index += (1 << 7)
              if item == 'ARQUEJO': index += (1 << 8)
              if item == 'ATONIA': index += (1 << 9)
              if item == 'AUTENTICIDADE': index += (1 << 10)
              if item == 'AUTOCÍDIO': index += (1 << 11)
              if item == 'AUTOESTIMA': index += (1 << 12)
              if item == 'BELEZA': index += (1 << 13)
              if item == 'BONDADE': index += (1 << 14)
              if item == 'COMPLACÊNCIA': index += (1 << 15)
              if item == 'COMPULSÃO': index += (1 << 16)
              if item == 'CORAGEM': index += (1 << 17)
              if item == 'DECÊNCIA': index += (1 << 18)
              if item == 'DESAFETO': index += (1 << 19)
              if item == 'DESÂNIMO': index += (1 << 20)
              if item == 'DESARMONIA': index += (1 << 21)
              if item == 'DESTEMOR': index += (1 << 22)
              if item == 'DIGNIDADE': index += (1 << 23)
              if item == 'ELEGÂNCIA': index += (1 << 24)
              if item == 'EMPATIA': index += (1 << 25)
              if item == 'ENGODO': index += (1 << 26)
              if item == 'ESPERANÇA': index += (1 << 27)
              if item == 'ESTIMA': index += (1 << 28)
              if item == 'ESTREITEZA': index += (1 << 29)
              if item == 'EUFORIA': index += (1 << 30)
              if item == 'FORTÚNIO': index += (1 << 31)
              if item == 'FRACASSO': index += (1 << 32)
              if item == 'FRAQUEZA': index += (1 << 33)
              if item == 'FUGAZ': index += (1 << 34)
              if item == 'GENTILEZA': index += (1 << 35)
              if item == 'GRATIDÃO': index += (1 << 36)
              if item == 'HARMONIA': index += (1 << 37)
              if item == 'HUMILDADE': index += (1 << 38)
              if item == 'IMPAVIDEZ': index += (1 << 39)
              if item == 'IMPOTÊNCIA': index += (1 << 40)
              if item == 'IMPULSIVIDADE': index += (1 << 41)
              if item == 'INAPETÊNCIA': index += (1 << 42)
              if item == 'INDIFERENÇA': index += (1 << 43)
              if item == 'INDULGÊNCIA': index += (1 << 44)
              if item == 'INQUIETUDE': index += (1 << 45)
              if item == 'INTELIGÊNCIA': index += (1 << 46)
              if item == 'LETARGIA': index += (1 << 47)
              if item == 'MÁGOA': index += (1 << 48)
              if item == 'MANIA': index += (1 << 49)
              if item == 'MAQUIAVÉLICO': index += (1 << 50)
              if item == 'MELANCOLIA': index += (1 << 51)
              if item == 'MORTE': index += (1 << 52)
              if item == 'NOSTALGIA': index += (1 << 53)
              if item == 'OBCECAÇÃO': index += (1 << 54)
              if item == 'OBSESSÃO': index += (1 << 55)
              if item == 'PERECÍVEL': index += (1 << 56)
              if item == 'PERSISTÊNCIA': index += (1 << 57)
              if item == 'PREOCUPAÇÃO': index += (1 << 58)
              if item == 'PROSTRAÇÃO': index += (1 << 59)
              if item == 'PRUDÊNCIA': index += (1 << 60)
              if item == 'RUMINAÇÃO': index += (1 << 61)
              if item == 'RAIVA': index += (1 << 62)
              if item == 'RANCOR': index += (1 << 63)
              if item == 'SATISFAÇÃO': index += (1 << 64)
              if item == 'SIGILO': index += (1 << 65)
              if item == 'SOLIDÃO': index += (1 << 66)
              if item == 'SUICÍDIO': index += (1 << 67)
              if item == 'TENACIDADE': index += (1 << 70)
              if item == 'VIRTUDE': index += (1 << 71)                                                                    
            appendData(dataResp, 'Resp_124', index)                                                                         
            appendData(dataResp, 'Elipsed_124', timeElipsed)                 
        case 69: # + 10
            for index,item in enumerate(colData.split(";")):
                item = remover_acentos_e_transformar_minusculo(item) 
                match index:
                    case 0: # 
                        replace_with_other(dataResp,125 + index,item,[
                            ('^barro',"0.0"),
                            ('barro', "1.0")                 
                        ])
                    case 1: # 
                        replace_with_other(dataResp,125 + index,item,[
                            ('^brinquedo',"0.0"),                           
                            ('brinquedo', "1.0")           
                        ])
                    case 2: # 
                        replace_with_other(dataResp,125 + index,item,[
                            ('^sol',"0.0"),                          
                            ('sol', "1.0")   
                        ])                      
                    case 3: # 
                        replace_with_other(dataResp,125 + index,item,[
                            ('^medo',"0.0"),
                            ('medo', "1.0")
                        ])                       
                    case 4: # 
                        replace_with_other(dataResp,125 + index,item,[                            
                            ('^rio',"0.0"),
                            ('rio', "1.0")                         
                        ])
                    case 5: # 
                        replace_with_other(dataResp,125 + index,item,[
                            ('^luxo',"0.0"),                            
                            ('luxo', "1.0")
                        ]) 
                        
                    case 6: # 
                        replace_with_other(dataResp,125 + index,item,[
                            ('^grande',"0.0"),                            
                            ('grande', "1.0")                           
                        ]) 
                    case 7: # 
                        replace_with_other(dataResp,125 + index,item,[
                            ('^caderno',"0.0"),                            
                            ('caderno', "1.0")                            
                        ]) 
                    case 8: # 
                        replace_with_other(dataResp,125 + index,item,[
                            ('^cadeira',"0.0"),                            
                            ('cadeira', "1.0")
                        ])                                                        
                    case 9: # 
                        replace_with_other(dataResp,125 + index,item,[
                            ('^escola',"0.0"),                            
                            ('escola', "1.0")                           
                        ])                                                                                                                                                                                
                appendData(dataResp, f'Elipsed_{125 + index}', timeElipsed)                                                                                              
        case 70: # + 8
            colData = re.sub(r'<optimized out>#\w+\((Sim|Não)\)', r'\1', colData)       
            for index,item in enumerate(colData.split(";")):
                replace_with_other(dataResp,135 + index,item,[
                    ('Não', "0.0"),
                    ('Sim', "1.0"),
                ])                                    
                appendData(dataResp, f'Elipsed_{135 + index}', timeElipsed)  
        case 71: # + 5
            for index,item in enumerate(colData.split(";")):
                replace_with_other(dataResp,143 + index,item,[
                    ('Nunca', "0.0"),
                    ('Uma vez por mês ou menos (raramente)', "0.25"),
                    ('Duas a quatro vezes ao mês (às vezes)', "0.5"),
                    ('Duas a três vezes por semana', "0.75"),
                    ('Na maioria dos dias ou sempre', "1.0")                        
                ])                                                                                   
                appendData(dataResp, f'Elipsed_{143 + index}', timeElipsed)  
        case 72: # + 7
            colData = re.sub(r'<optimized out>#\w+\((Sim|Não)\)', r'\1', colData)       
            for index,item in enumerate(colData.split(";")):
                replace_with_other(dataResp,148 + index,item,[
                    ('Não', "0.0"),
                    ('Sim', "1.0"),
                ])                                    
                appendData(dataResp, f'Elipsed_{148 + index}', timeElipsed)   
        case 73: # Daltonismo é o termo usado para denominar a falta de sensibilidade
            replace_with_other(dataResp,155,colData,[
              ("Tenho daltonismo", "0.0"),
              ("Não tenho daltonismo", "1.0")        
            ])           
            appendData(dataResp, 'Elipsed_155', timeElipsed)
            timeAtual.day_of_week            
        # case 74: # 
        #     if float(dataResp['Resp_156']) == 1.0:
        #         replace_with_other(dataResp,154,colData,[
        #         ("Tenho daltonismo", "0.0"),
        #         ("Não tenho daltonismo", "1.0")        
        #         ])           
        #         appendData(dataResp, 'Elipsed_156', timeElipsed)
        #     else:                
        #         replace_with_other(dataResp,156,colData,[
        #         ("Segunda-feira","1.0" if 0 == timeAtual.day_of_week else "0.0"),
        #         ("Terça-feira", "1.0" if 1 == timeAtual.day_of_week else "0.0"),
        #         ("Quarta-feira", "1.0" if 2 == timeAtual.day_of_week else "0.0"),
        #         ("Quinta-feira", "1.0" if 3 == timeAtual.day_of_week else "0.0"), 
        #         ("Sexta-feira", "1.0" if 4 == timeAtual.day_of_week else "0.0"),
        #         ("Sábado", "1.0" if 5 == timeAtual.day_of_week else "0.0"),
        #         ("Domingo", "1.0" if 6 == timeAtual.day_of_week else "0.0")                                     
        #         ])           
        #         appendData(dataResp, 'Elipsed_156', timeElipsed)
        # case 75: # 
        #     if float(dataResp['Resp_154']) == 1.0:
        #         replace_with_other(dataResp,154,colData,[
        #         ("Tenho daltonismo", "0.0"),
        #         ("Não tenho daltonismo", "1.0")        
        #         ])           
        #         appendData(dataResp, 'Elipsed_154', timeElipsed)
        #     else:
        #         replace_with_other(dataResp,154,colData,[
        #         ("Tenho daltonismo", "0.0"),
        #         ("Não tenho daltonismo", "1.0")        
        #         ])           
        #         appendData(dataResp, 'Elipsed_154', timeElipsed)                                                                                                                                                                                                                                                                                                                                                                              
        # case _:
        #     appendData(dataResp, f'Resp_N{72 + colNum}', re.sub(r'<optimized out>#\w+\((Sim|Não)\)', r'\1', colData))
        #     appendData(dataResp, f'Elipsed_N{72 + colNum}', timeElipsed)                
        

In [ ]:
# Leitura do arquivo xlsx usando pandas
df = pd.read_excel(file_path, sheet_name='BDados')

# Inicializa variáveis necessárias
dataResp = {}
count = 0
timeAnterior = []  # Inicializa timeAnterior como uma lista vazia

# Loop para processar colunas que começam com "Tela"
for indexCol, colStr in enumerate(df.columns):

    if colStr.startswith("Tela") and df.iloc[0, indexCol] != "" and pd.notna(df.iloc[0, indexCol]):
        timeAtual = []
        timeElipsed = []
        value = []
        
        colNum = int(re.search(r"Tela (\d+)", colStr).group(1))
        
        for indexRow in range(df.shape[0]):
            cell_value = str(df.iloc[indexRow, indexCol])
            match = re.match(r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}) - (.+)", cell_value)
            
            if match:
                timeAtual.append(pd.to_datetime(match.group(1)))
                if colNum != 1:                                    
                    trataCol(dataResp, colNum, match.group(2), timeAtual[-1], timeAnterior[indexRow]) # globals()[f"func_tela_colNum"](match.group(2))
            else:
                print("Ocorreu um erro ao processar a célula:", cell_value)
        
        timeAnterior = timeAtual 
        
# Cria o novo DataFrame com os dados processados
df_novo = pd.DataFrame(dataResp)

# Usando ExcelWriter para adicionar a nova aba ao arquivo Excel existente com if_sheet_exists='replace'
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_novo.to_excel(writer, sheet_name='TDados', index=False)